In [1]:
from time import time
t0 = time()

In [2]:
import os

import geopandas as gpd
import pandas as pd
import numpy as np
import momepy as mm
import networkx as nx

from sqlalchemy import create_engine

import matplotlib.pyplot as plt

user = os.environ.get('DB_USERNAME')
pwd = os.environ.get('DB_PASSWORD')
host = os.environ.get('DB_HOSTNAME')
port = os.environ.get('DB_PORT')

url = f"postgres+psycopg2://{user}:{pwd}@{host}:{port}/geodemo"
engine = create_engine(url)

In [3]:
rlwys = gpd.read_postgis('SELECT * FROM railways', engine, geom_col='geometry')

In [4]:
rivers = gpd.read_postgis('SELECT * FROM rivers', engine, geom_col='geometry')

In [5]:
rivers.form.unique()

array(['inlandRiver', 'tidalRiver', 'lake', 'canal'], dtype=object)

In [6]:
regions = gpd.read_postgis('SELECT * FROM country_region', engine, geom_col='geometry')
england = regions.loc[regions.Name=='England']

In [9]:
roads = gpd.read_postgis('SELECT * FROM openroads', engine, geom_col='geometry')

In [10]:
roads.roadFunction.unique()

array(['Minor Road', 'Local Road', 'Restricted Local Access Road',
       'A Road', 'Local Access Road', 'Motorway', 'Secondary Access Road',
       'B Road'], dtype=object)

In [11]:
mwys = roads.loc[roads.roadFunction=='Motorway']

In [12]:
a_rd = roads.loc[roads.roadFunction=='A Road']

In [13]:
# not borders
#b_rd = roads.loc[roads.roadFunction=='B Road']
#l_rd = roads.loc[roads.roadFunction=='Local Road']

In [14]:
borders = pd.concat([mwys, a_rd, rlwys, rivers])

In [15]:
border_union = borders.unary_union

In [ ]:
border_union

In [8]:
eng_border = england.geometry.boundary

eng = gpd.GeoDataFrame(geometry=eng_border)

In [16]:
from shapely.ops import polygonize

In [17]:
polygons = polygonize(border_union)

In [18]:
enclosures = gpd.array.from_shapely(list(polygons), crs=roads.crs)

In [19]:
enclosures

<GeometryArray>
 ...
Length: 58410, dtype: geometry

In [20]:
gpd.GeoDataFrame(geometry=enclosures, crs=roads.crs).to_parquet('../data/enclosures.pq')

<ipython-input-20-7f1fa0a76a8d>:1: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a production setting unless you are able to rewrite your Parquet/Feather files.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')
  gpd.GeoDataFrame(geometry=enclosures, crs=roads.crs).to_parquet('../data/enclosures.pq')


In [21]:
gdf = gpd.read_parquet('../data/enclosures.pq')

In [22]:
gdf.to_postgis('enclosures', engine, if_exists='replace')

In [23]:
t = time()
print(t-t0)

314.38188767433167


In [ ]:
print(f'{(t-t0)//60} minutes, {(t-t0)%60} seconds')